## Algorithmic Trading Framework
Data is from testnet, and therefore not accurate.

Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [48]:
import os
import pandas as pd
import numpy as np
# import scipy.optimize as sco
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
# pd.reset_option('display.float_format')

In [49]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
pd.to_datetime(client.time()['serverTime'], unit='ms')

Timestamp('2022-05-09 22:23:21.635000')

If your time is off you need to synchronize with timeserver

In [50]:
balance = pd.json_normalize(client.account()['balances'])
balance

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [51]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]

In [52]:
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols)
assets

BTCUSDT                                                   \
       Open time            Open             High             Low   
0  1651622400000  38127.16000000   67952.01000000   9000.00000000   
1  1651708800000  39690.01000000   48256.09000000  12000.00000000   
2  1651795200000  36552.97000000   90000.00000000   9500.00000000   
3  1651881600000  35851.70000000  179685.00000000   2000.00000000   
4  1651968000000  35472.40000000  172120.00000000   6988.00000000   
5  1652054400000  34036.87000000   44600.00000000   6392.00000000   

                                                                     \
            Close         Volume     Close time  Quote asset volume   
0  39695.80000000  3289.47888900  1651708799999  128626868.06394152   
1  36551.04000000  3747.85911700  1651795199999  142179013.42520536   
2  35844.54000000  5133.87878000  1651881599999  185618082.67083564   
3  35472.39000000  2648.74837000  1651967999999   94693433.70235498   
4  34035.00000000  5928.73934500  1652054399999  204738551.12075512   
5  30999.86000000  7095.92520200  1652140799999  231746516.81966914   

                                                ...     XRPUSDT              \
  Number of trades Taker buy base asset volume  ...        High         Low   
0           109403               2070.21409400  ...  0.64990000  0.60830000   
1           136426               2305.80703300  ...  0.65710000  0.58040000   
2           195320               2791.12783400  ...  0.61630000  0.58130000   
3           110202               1439.33242400  ...  0.60340000  0.56940000   
4           216686               3380.01233600  ...  0.58280000  0.55810000   
5           239520               4368.86833300  ...  0.57870000  0.47290000   

                                                                    \
        Close             Volume     Close time Quote asset volume   
0  0.64610000   7188003.90000000  1651708799999   4497422.00637000   
1  0.59830000  11682556.70000000  1651795199999   7147202.83414000   
2  0.58130000  12824794.10000000  1651881599999   7706389.17155000   
3  0.58180000   4378006.60000000  1651967999999   2579982.36840000   
4  0.56620000   9985261.30000000  1652054399999   5708676.23890000   
5  0.50590000  25126656.40000000  1652140799999  13216103.04203000   

                                                                             \
  Number of trades Taker buy base asset volume Taker buy quote asset volume   
0             8638            7020708.90000000             4393175.81487000   
1            13976           11566933.90000000             7074742.84037000   
2            15203           12610238.80000000             7578080.05335000   
3             5475            4097615.10000000             2413970.94234000   
4            10831            9690239.70000000             5539919.86459000   
5            26308           24761300.40000000            13023473.30224000   

          
  Ignore  
0      0  
1      0  
2      0  
3      0  
4      0  
5      0  

[6 rows x 72 columns]

Formula to convert from unix time to datetime, needs to be set as index

In [53]:
pd.concat([pd.to_datetime(assets[symbol, 'Close time'], unit='ms') for symbol in symbols], axis=1)

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
,Close time,Close time,Close time,Close time,Close time,Close time
0,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999
1,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999
2,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999
3,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999
4,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999
5,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999


Swapping levels for easier selection 

In [54]:
assets = assets.swaplevel(axis=1)

## Returns

In [55]:
assets_close = assets["Close"].copy().astype(float) # Daily close prices
assets_close

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
0,39695.80,2940.67,402.60,106.4,0.08630,0.6461
1,36551.04,2747.93,378.50,96.9,0.07454,0.5983
2,35844.54,2655.33,157.07,95.6,0.08167,0.5813
3,35472.39,2637.33,365.50,94.5,0.08349,0.5818
4,34035.00,2519.51,356.00,94.0,0.08795,0.5662
5,30999.86,2291.26,309.10,79.9,0.07576,0.5059


In [56]:
returns = assets_close.pct_change().dropna()

In [57]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
1,-0.079221,-0.065543,-0.059861,-0.089286,-0.136269,-0.073982
2,-0.019329,-0.033698,-0.585020,-0.013416,0.095653,-0.028414
3,-0.010382,-0.006779,1.326988,-0.011506,0.022285,0.000860
4,-0.040521,-0.044674,-0.025992,-0.005291,0.053420,-0.026813
5,-0.089177,-0.090593,-0.131742,-0.150000,-0.138601,-0.106499


In [58]:
returns.mean(axis = 0)

BTCUSDT   -0.047726
ETHUSDT   -0.048257
BNBUSDT    0.104875
LTCUSDT   -0.053900
TRXUSDT   -0.020703
XRPUSDT   -0.046970
dtype: float64

## Annualised Risk & Return

In [59]:
def annualised_risk_return(returns):
    stats = returns.agg(['mean', 'std']).T
    stats.columns = ['Return', 'Risk']
    stats.Return = stats.Return*365.25 # Crypto exchanges trade every day, including leap days.
    stats.loc[stats.Return < -1, 'Return'] = -1 # Set losses > 100% to -100%
    stats.Risk = stats.Risk * np.sqrt(365.25)
    return stats 

In [63]:
stats = annualised_risk_return(returns) # Risk may not be accurate for mean daily losses > 0.274% ()

## Sharpe

In [61]:
riskfree_return = 0.031 # 5 Year Treasury Rate, but testnet resets every month

In [62]:
stats['Sharpe'] = stats['Return'].sub(riskfree_return)/stats['Risk']
stats

,Return,Risk,Sharpe
BTCUSDT,-1.000000,0.673189,-1.531517
ETHUSDT,-1.000000,0.607101,-1.698234
BNBUSDT,38.305512,13.747565,2.784094
LTCUSDT,-1.000000,1.219488,-0.845437
TRXUSDT,-1.000000,2.096534,-0.491764
XRPUSDT,-1.000000,0.817102,-1.261776


## Optimal Sharpe Ratio Portfolio

In [64]:
asset_qty = len(assets_close.columns)

In [66]:
def portfolio_return(weights):
    return returns.dot(weights.T).mean() * 365.25

In [67]:
def portfolio_risk(weights):
    return returns.dot(weights.T).std() * 365.25

In [68]:
def minimized_sharpe(weights):
    return (riskfree_return - portfolio_return(weights))/portfolio_risk(weights)

In [69]:
equal_weights = np.full(asset_qty, 1/asset_qty)

In [70]:
constraint = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1})

In [71]:
bounds = tuple((0,1) for x in range(asset_qty))

In [72]:
# Sequential Least Squares Programming
optimum = sco.minimize(minimized_sharpe, equal_weights, method='SLSQP', bounds=bounds, constraints=constraint)

In [73]:
optimum # No surprise with that 157.07$ BNB outlier

     fun: -0.14567625518855917
     jac: array([ 0.06917026,  0.07081947, -0.00011799,  0.07817384,  0.02944852,
        0.06962823])
 message: 'Optimization terminated successfully'
    nfev: 14
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0., 0., 1., 0., 0., 0.])

In [74]:
optimal_weights = optimum['x']
optimal_weights

array([0., 0., 1., 0., 0., 0.])

In [75]:
portfolio_return(optimal_weights)

38.30551214338975

In [76]:
portfolio_risk(optimal_weights)

262.73679326701745

In [77]:
-minimized_sharpe(optimal_weights)

0.14567625518855917